In [ ]:
import sys
sys.path.append("../../")

from cnstats.stats import stats
from macroquant.dataapi.cnstats import *
from datetime import datetime, timedelta

In [78]:
def profit_df(datestr):
    # print(datestr)
    df = cnstats_industry_profit(datestr)
    df2 = cnstats_industry_employment(datestr)
    if len(df2) > 0:
        df = df.join(df2, on='industry', how='inner')
    if len(df) > 0:
        df = df.rename({'利润总额': f'利润总额_{datestr}', '平均用工人数': f'平均用工人数_{datestr}'})
        df = df.with_columns(
            (pl.col(f'利润总额_{datestr}')*10000/pl.col(f'平均用工人数_{datestr}')).alias(f'人均创造利润_{datestr}'),
        )
    return df

In [ ]:
start_date, end_date = '202410', '202510'

dates = []
date = start_date
while(date <= end_date):
    dates.append(date)
    dt = datetime.strptime(date, '%Y%m')
    dt = dt + timedelta(days=31)
    if dt.month == 1:
        dt = dt + timedelta(days=31)
    date = dt.strftime('%Y%m')
print(dates)
df = pl.DataFrame()
for date in dates:
    d = profit_df(date)
    if len(d) > 0:
        if len(df) > 0:
            df = df.join(d, on='industry', how='inner')
        else:
            df = d

profit_columns = [f'利润总额_{date}' for date in dates]
avg_columns = [f'平均用工人数_{date}' for date in dates]
df = df.with_columns(
    pl.sum_horizontal(profit_columns).alias("利润总额"),
    (pl.sum_horizontal(avg_columns)/len(dates)).alias("平均用工人数"),
)

df = df.with_columns(
        [
         (pl.col(f'利润总额')*10000/pl.col(f'平均用工人数')).alias(f'人均创造利润'),
         ((pl.col(f'利润总额_{end_date}')-pl.col(f'利润总额_{start_date}'))/pl.col(f'利润总额_{start_date}')).alias(f'利润增长率'),
         ((pl.col(f'人均创造利润_{end_date}')-pl.col(f'人均创造利润_{start_date}'))/pl.col(f'人均创造利润_{start_date}')).alias(f'人均创造利润增长率'),
        ]
    )
df.select(pl.col(['industry', '利润总额', '人均创造利润', '利润增长率', '人均创造利润增长率']))


['202410', '202411', '202412', '202502', '202503', '202504', '202505', '202506', '202507', '202508', '202509', '202510']


industry,利润总额,人均创造利润,利润增长率,人均创造利润增长率
str,f64,f64,f64,f64
"""采矿业""",70362.1,178.015475,-0.276477,-0.252372
"""煤炭开采和洗选业""",30261.4,119.851084,-0.492176,-0.478528
"""石油和天然气开采业""",26034.8,526.132705,-0.126955,-0.118101
"""黑色金属矿采选业""",3443.4,149.713043,-0.221452,-0.183725
"""有色金属矿采选业""",7931.3,337.262934,0.327521,0.299632
…,…,…,…,…
"""金属制品、机械和设备修理业""",1346.0,50.239502,-0.022121,-0.120968
"""电力、热力、燃气及水生产和供应业""",60661.5,180.084607,0.108485,0.124713
"""电力、热力生产和供应业""",51325.0,210.197604,0.139655,0.161292


In [ ]:
df.select(pl.col(['industry', '利润总额', '人均创造利润', '利润增长率', '人均创造利润增长率'])).write_csv('data/行业利润增长率_过去1年.csv')
